<a href="https://colab.research.google.com/github/288756/VisArtificial/blob/master/Miniproyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive

# Montar el Google Drive en el directorio del proyecto y descomprimir el fichero con los datos
drive.mount('/content/gdrive')
!unzip -n '/content/gdrive/My Drive/vision-artificial.zip' >> /dev/null  # ACTUALIZAR: ruta al fichero comprimido


Mounted at /content/gdrive


In [40]:
from sklearn.preprocessing import LabelEncoder

# Especificar las rutas al directorio con las imágenes y al fichero con las etiquetas
data_path = '/content/'
imgtrain_dir = data_path + "train/images/"
csvtrain_file = data_path + "train.csv"

imgtest_dir = data_path + "test/images/"
csvtest_file = data_path + "test.csv"
# Leer el fichero CSV con las etiquetas

dftrain = pd.read_csv(csvtrain_file, dtype = {"class": "category"})
dftest = pd.read_csv(csvtest_file, dtype = {"class": "category"})
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dftest = dftest.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
# Codificar las etiquetas
label_encoder = LabelEncoder()
dftrain['class'] = label_encoder.fit_transform(dftrain['class'])
dftrain

,image_filename,mask_filename,class
0,image_426.png,mask_426.png,1
1,image_433.png,mask_433.png,0
2,image_184.png,mask_184.png,1
3,image_061.png,mask_061.png,2
4,image_357.png,mask_357.png,2
...,...,...,...
332,image_337.png,mask_337.png,1
333,image_096.png,mask_096.png,0
334,image_031.png,mask_031.png,0
335,image_381.png,mask_381.png,0


In [41]:
val_size = int(len(dftrain) * 0.2)
dftrain = dftrain.sample(frac=1).reset_index(drop=True)  # barajar el dataframe
dfval = dftrain[:val_size]
print(f'Número de ejemplos del conjunto de entrenamiento: {dftrain.shape[0]}')
print(f'Número de ejemplos del conjunto de validación: {dfval.shape[0]}')
print(f'Número de ejemplos del conjunto de test: {dftest.shape[0]}')
dftrain = dftrain.reset_index(drop=True)
dfval = dfval.reset_index(drop=True)

Número de ejemplos del conjunto de entrenamiento: 337
Número de ejemplos del conjunto de validación: 67
Número de ejemplos del conjunto de test: 113


In [56]:
import tensorflow as tf
from tensorflow.data import Dataset

# Dimensiones deseadas de la imagen
img_width, img_height = 224, 224
n_channels = 3                # número de canales (RGB)
n_classes = 3                 # número de clases
x_col = 'image_filename'      # nombres de las columnas en el fichero CSV
y_col = 'class'

# Cargar y preprocesar imágenes
def load_and_preprocess_image(image_filename, label_str):
    image_path = tf.strings.join([imgtrain_dir, image_filename])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=n_channels)
    # Redimensionar la imagen al tamaño deseado con relleno de ceros si es necesario
    image = tf.image.resize_with_pad(image, img_width, img_height)
    image = image / 255.0                               # normalización
    label = label_str                                     # No es necesario convertir a número si las etiquetas son strings
    return image, label

# Crear conjunto de datos
def get_dataset(df):
    image_filenames = df[x_col].values
    labels = df[y_col].values
    dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
    dataset = dataset.map(load_and_preprocess_image)
    return dataset

# Crear los conjuntos de datos y preparar los lotes
batch_size = 128
train_dataset = get_dataset(dftrain).batch(batch_size)
val_dataset = get_dataset(dfval).batch(batch_size)

print(f'Número de lotes del conjunto de entrenamiento: {len(train_dataset)}')
print(f'Número de lotes del conjunto de validación: {len(val_dataset)}')

Número de lotes del conjunto de entrenamiento: 3
Número de lotes del conjunto de validación: 1


In [57]:
import tensorflow as tf
from tensorflow.keras import layers, models

def simple_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define the input shape (width, height, channels)
input_shape = (224, 224, 3)
# Define the number of classes
num_classes = 3  # Assuming you have 3 classes

# Create the model
model = simple_cnn_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use this loss function for integer labels
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 flatten_2 (Flatten)         (None, 173056)           

In [58]:
import numpy as np

# Entrenar el modelo con los datos preparados previamente
history = model.fit(train_dataset,
          epochs=6,   # número de epochs
          verbose=2,  # muestra información al finalizar cada epoch
          validation_data=val_dataset)

# Imprimir el error mínimo de entrenamiento y validación
train_trace = np.array(history.history['loss'])
print(f'\nError mínimo en entrenamiento: {min(train_trace):.6f}')

val_trace = np.array(history.history['val_loss'])
print(f'Error mínimo en validación: {min(val_trace):.6f}')

Epoch 1/6
3/3 - 38s - loss: 6.2511 - accuracy: 0.3561 - val_loss: 1.8168 - val_accuracy: 0.4627 - 38s/epoch - 13s/step
Epoch 2/6
3/3 - 35s - loss: 1.4723 - accuracy: 0.4065 - val_loss: 1.0052 - val_accuracy: 0.4776 - 35s/epoch - 12s/step
Epoch 3/6
3/3 - 36s - loss: 1.0074 - accuracy: 0.4926 - val_loss: 0.9243 - val_accuracy: 0.4776 - 36s/epoch - 12s/step
Epoch 4/6
3/3 - 37s - loss: 0.9974 - accuracy: 0.4955 - val_loss: 0.9049 - val_accuracy: 0.4776 - 37s/epoch - 12s/step
Epoch 5/6
3/3 - 34s - loss: 0.9337 - accuracy: 0.5074 - val_loss: 0.8482 - val_accuracy: 0.5075 - 34s/epoch - 11s/step
Epoch 6/6
3/3 - 34s - loss: 0.8861 - accuracy: 0.6202 - val_loss: 0.7703 - val_accuracy: 0.6567 - 34s/epoch - 11s/step

Error mínimo en entrenamiento: 0.886096
Error mínimo en validación: 0.770313


In [59]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input

# Cargar el modelo previamente entrenado
model = get_model()  # Suponiendo que ya has definido y entrenado el modelo

# Directorio donde se encuentran las imágenes de prueba
test_images_dir = 'test/images/'

# Obtener la lista de nombres de archivos de las imágenes de prueba
test_filenames = os.listdir(test_images_dir)

# Crear una lista para almacenar las predicciones
predictions = []

# Iterar sobre cada imagen de prueba
for filename in test_filenames:
    # Cargar la imagen y preprocesarla
    img_path = os.path.join(test_images_dir, filename)
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión adicional para el lote

    # Realizar la predicción
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)  # Obtener la clase predicha
    predictions.append(predicted_class)

# Crear un DataFrame con los nombres de los archivos y las predicciones
df_predictions = pd.DataFrame({'image_filename': test_filenames, 'predicted_class': predictions})

# Guardar el DataFrame en un archivo CSV
df_predictions.to_csv('test_predictions.csv', index=False)

# Imprimir las primeras filas del DataFrame para verificar
print(df_predictions.head())


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   